In [3]:
print ("pratip")

pratip


In [62]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

train=pd.read_csv("banktraindata.csv")

test=pd.read_csv("banktestdata.csv")

train['type'] ='Train'

test['type'] = 'Test'

fullData = pd.concat([test,train] , axis=0, sort=False )

fullData

ID_col = ['ID']

target_col = ["subscribed"]

cat_cols = ['job','marital','education','default','housing','loan','contact','month', 'poutcome']

num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col)-set(other_col))

other_col=['type'] #Test and Train Data set identifier

fullData.isnull().any()

num_cat_cols = num_cols+cat_cols

count=0
for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        count= count + 1
        fullData[var+'_NA']=fullData[var].isnull()*1 

#print(count)        
#Impute numerical missing values with mean

fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean(),inplace=True)

#Impute categorical missing values with -9999

fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)
#print(fullData[cat_cols])
#create label encoders for categorical features
flag=0
for var in cat_cols: 
    number = LabelEncoder()
    flag = flag +1
    fullData[var] = number.fit_transform(fullData[var].astype('str'))

#print(flag)    
#Target variable is also a categorical so convert it
fullData["subscribed"] = number.fit_transform(fullData["subscribed"].astype('str'))
#print(fullData["subscribed"])


train=fullData[fullData['type']=='Train']
test=fullData[fullData['type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col))
#print(features)

x_train = Train[list(features)].values

#converting numpy array into panda dataframe so that we can fill the missing value
x_train = pd.DataFrame(x_train)
x_train=pd.DataFrame(x_train).fillna(0)

#print(x_train.head())

#again to numpy
x_train = x_train.to_numpy()
print(x_train[1])



y_train = Train["subscribed"].values
print(y_train[1])
print(y_train[1])


x_validate = Validate[list(features)].values

#converting numpy array into panda dataframe so that we can fill the missing value
x_validate= pd.DataFrame(x_validate)
x_validate=x_validate.fillna(0)
#print(x_validate.head())

#again to numpy
x_validate = x_validate.to_numpy()
print(x_validate[1])

y_validate = Validate["subscribed"].values
print(y_validate)
print(y_validate[9])

x_test=test[list(features)].values

#converting numpy array into panda dataframe so that we can fill the missing value
x_test = pd.DataFrame(x_test)
x_test=x_test.fillna(0)

#again to numpy
x_test = x_test.to_numpy()
print(x_test[1])

random.seed(100)

rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1],pos_label=2) #pos_label is to tell that 2 is true and 1 is false in subscribed
roc_auc = auc(fpr, tpr)
print (roc_auc)

final_status = rf.predict_proba(x_test)
test["subscribed"]=final_status[:,1]
test.to_csv('model_output2.csv',columns=['REF_NO','subscribed'])

[0 0 2 0 0 0 0 4 0 0 2 0 0 0 6 0]
2
2
[1 0 3 0 0 0 1 7 0 0 1 0 0 0 5 0]
[1 1 1 ... 2 1 2]
2
[0 0 3 0 1 0 0 5 0 0 0 0 0 0 5 0]
0.2943227191556813


C:\Users\Pratip-Zalak\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
